In [1]:
import cv2
import numpy as np
import os
import sys
from samples import coco
from mrcnn import utils
from mrcnn import model as modellib
# https://github.com/tensorflow/tensorflow/issues/24828
# Error : Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
for physical_device in physical_devices: 
    tf.config.experimental.set_memory_growth(physical_device, True)

Using TensorFlow backend.


In [2]:
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None

In [3]:
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params) 
        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
    except Exception as e:
        print(e)

In [4]:
ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [5]:
class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    
    IMAGES_PER_GPU = 1

config = InferenceConfig()

In [6]:
model = modellib.MaskRCNN(
    mode="inference", model_dir=MODEL_DIR, config=config)
model.load_weights(COCO_MODEL_PATH, by_name=True)
class_names = [
    'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
    'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
    'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
    'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
    'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
    'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
    'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
    'teddy bear', 'hair drier', 'toothbrush' 
]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [7]:
def apply_mask(image, mask):
    image[:, :, 0] = np.where(
        mask == 0,
        0,
        image[:, :, 0]
    )
    image[:, :, 1] = np.where(
        mask == 0,
        0,
        image[:, :, 1]
    )
    image[:, :, 2] = np.where(
        mask == 0,
        0,
        image[:, :, 2]
    )
    return image

In [8]:
def display_instances(image, boxes, masks, ids, names, scores):
    # max_area will save the largest object for all the detection results
    max_area = 0
    mask = None
    
    # n_instances saves the amount of all objects
    n_instances = boxes.shape[0]

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        if not np.any(boxes[i]):
            continue

        # compute the square of each object
        y1, x1, y2, x2 = boxes[i]
        square = (y2 - y1) * (x2 - x1)

        # use label to select person object from all the 80 classes in COCO dataset
        label = names[ids[i]]
        if label == 'car':
            # save the largest object in the image as main character
            # other people will be regarded as background
            if square > max_area:
                max_area = square
                mask = masks[:, :, i]
            else:
                continue
        else:
            continue
    color, edge = make_cartoon(image)
    if mask is not None:
        color = apply_mask(color, mask)
        edge = apply_mask(edge , mask)
        return color, edge
    else:
        print("Mask is None")
        return color, edge
    

In [9]:
def make_cartoon(img_rgb):
    numDownSamples = 2
    numBilateralFilters = 7
    
    img_color = img_rgb
    for i in range(numDownSamples):
        img_color = cv2.pyrDown(img_color)
    for i in range(numBilateralFilters):
        img_color = cv2.bilateralFilter(img_color, 9, 9, 7)
    for i in range(numDownSamples):
        img_color = cv2.pyrUp(img_color)
    # pyrUp Down 시, size가 2로 나눠지지 않을 때 사이즈 변화할 수 있음
    img_color = cv2.resize(img_color, dsize=(img_rgb.shape[1], img_rgb.shape[0]), interpolation=cv2.INTER_LINEAR)
    
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    img_blur = cv2.medianBlur(img_gray, 3)
    img_edge = cv2.Canny(img_blur, 50, 150, apertureSize=3)
    # img_edge2 = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5 , 2)

    kernel = np.ones((3,3), dtype=np.float) / 12.0
    img_edge = cv2.filter2D(img_edge, 0, kernel)
    img_edge = cv2.threshold(img_edge, 50, 255, 0)[1]
    
    #cartoon_color = cv2.subtract(img_color, img_edge)
    
    img_edge = cv2.bitwise_not(img_edge)
    img_edge = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2RGB)
    # img_edge2 = cv2.cvtColor(img_edge2, cv2.COLOR_GRAY2RGB)
    
    cartoon_color = cv2.bitwise_and(img_color, img_edge)
    # color2 = cv2.bitwise_and(img_color, img_edge2) 
    
    return cartoon_color, img_edge

In [10]:
def image_merge(img1, img2):
    vis = np.concatenate((img1, img2), axis=1)
    '''
    h1, w1 = img.shape[:2]
    h2, w2 = img.shape[:2]
    
    vis = np.zeros((max(h1, h2), w1+w2, 3), np.uint8)
    
    vis[:h1], :w1, :3] = img1
    vis[:h2, wa;w1+w2, :3] = img2
    '''
    return vis

In [13]:
output_dir = "output_foreground2/"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

input_dir = "car_dataset/"
input_path = os.path.join(input_dir, '*.jpg')
import glob
images = glob.glob(input_path)

In [14]:
num = 1
'''
c, e = make_cartoon(image)
cv2.imshow('c', c)
cv2.imshow('e', e)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

for image in images:
    image = cv2.imread(image)
    results = model.detect([image], verbose=0)
    r = results[0]
    cartoon_color, cartoon_edge = display_instances(
        image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
        )
    cartoon_color = cv2.resize(cartoon_color, dsize=(256, 256), interpolation=cv2.INTER_AREA)
    cartoon_edge = cv2.resize(cartoon_edge, dsize=(256, 256), interpolation=cv2.INTER_AREA)
    merged_image=image_merge(cartoon_color, cartoon_edge)
    imwrite(output_dir + "{}.jpg".format(num), merged_image)
    
    print("merged image {}.jpg".format(num), "is saved")
    print(cartoon_color.shape, cartoon_edge.shape, merged_image.shape)
    num+=1


merged image 1.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 2.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 3.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 4.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 5.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 6.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 7.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 8.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 9.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 10.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 11.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 12.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 13.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 14.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 15

merged image 117.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 118.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 119.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 120.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 121.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 122.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 123.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 124.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 125.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 126.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 127.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 128.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 129.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 130.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 

merged image 231.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 232.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 233.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 234.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
NO INSTANCES TO DISPLAY
Mask is None
merged image 235.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
Mask is None
merged image 236.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 237.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
Mask is None
merged image 238.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 239.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 240.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 241.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 242.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 243.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
m

merged image 344.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 345.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 346.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 347.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 348.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 349.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 350.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
Mask is None
merged image 351.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 352.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 353.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 354.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 355.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 356.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 357.jpg is saved
(256, 256, 3) (256, 2

merged image 458.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 459.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 460.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 461.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 462.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 463.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 464.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 465.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 466.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 467.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 468.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 469.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 470.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 512, 3)
merged image 471.jpg is saved
(256, 256, 3) (256, 256, 3) (256, 